- answer：
    - 关联规则中的支持度、置信度和提升度代表的什么，如何计算
        - 支持度： 项集{X,Y}在总项集里出现的概率。 $ Support=P(X&Y) $
        - 置信度：包含X的事务中同时包含Y事务的比例  $ Confidence=P(X&Y)/P(X）$
        - 提升度：包含X的事务中同时包含Y事务的比例”与“包含Y事务的比例”的比值 $ Lift=(P(X&Y)/P(X))/P(Y)=P(X&Y)/P(X)/P(Y) $
    - 关联规则与协同过滤的区别:
        - 协同过滤主要是以收集用户信息，通过协同依赖关系从而预测其他用户是否有相同特征等等，是 由大数据量到特征做 single prediction （ orient to features) 个性化的
        - 关联分析 是从大量数据关系中 获取一类到多类之间的关联性 ( orient to transactions) 整体的
    - 为什么我们需要多种推荐算法:
         - 多种推荐算法的融合 可以更好地解决单一算法的单边短板问题， 比如协同过滤有冷启动收集features等待的时间跨度， 或者 features 不明显，或者 数据存在大量 “噪声” 难以进行prediction时，就可采用其他方法进行补充混合。
        - 静态 / 动态 
    - 关联规则中的最小支持度、最小置信度该如何确定
        - 实际操作中，还是需要几次不同程度的tune值操作。总体而言，物料关联较小，人物关系关联较大，具体情况具体分析
    - 都有哪些常见的回归分析方法，评价指标是什么：
        - 主要有 A. 线性回归类   B. 岭回归   C. 弹性回归      evaluation : cost function  , MSE , RMSE . 

In [2]:
import pandas as pd
import numpy as np
data = pd.read_csv('Market_Basket_Optimisation.csv')
data

,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
0,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,low fat yogurt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,butter,light mayo,fresh bread,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7496,burgers,frozen vegetables,eggs,french fries,magazines,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7497,chicken,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7498,escalope,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#solution 1
data.fillna(0, inplace=True)
def get_transactions(df):
    transactions = []
    for i in range(0, len(df)):
        transactions.append([df.iloc[i,j] for j in range(0, df.shape[1]) if df.iloc[i,j] != 0])
    return transactions
trans = get_transactions(data)
trans

[['burgers', 'meatballs', 'eggs'],
 ['chutney'],
 ['turkey', 'avocado'],
 ['mineral water', 'milk', 'energy bar', 'whole wheat rice', 'green tea'],
 ['low fat yogurt'],
 ['whole wheat pasta', 'french fries'],
 ['soup', 'light cream', 'shallot'],
 ['frozen vegetables', 'spaghetti', 'green tea'],
 ['french fries'],
 ['eggs', 'pet food'],
 ['cookies'],
 ['turkey', 'burgers', 'mineral water', 'eggs', 'cooking oil'],
 ['spaghetti', 'champagne', 'cookies'],
 ['mineral water', 'salmon'],
 ['mineral water'],
 ['shrimp',
  'chocolate',
  'chicken',
  'honey',
  'oil',
  'cooking oil',
  'low fat yogurt'],
 ['turkey', 'eggs'],
 ['turkey',
  'fresh tuna',
  'tomatoes',
  'spaghetti',
  'mineral water',
  'black tea',
  'salmon',
  'eggs',
  'chicken',
  'extra dark chocolate'],
 ['meatballs', 'milk', 'honey', 'french fries', 'protein bar'],
 ['red wine', 'shrimp', 'pasta', 'pepper', 'eggs', 'chocolate', 'shampoo'],
 ['rice', 'sparkling water'],
 ['spaghetti', 'mineral water', 'ham', 'body spray',

In [5]:
#solution2
data = pd.read_csv('Market_Basket_Optimisation.csv',header=None, sep='|')[0].str.get_dummies(',')
data

,asparagus,almonds,antioxydant juice,asparagus,avocado,babies food,bacon,barbecue sauce,black tea,blueberries,...,turkey,vegetables mix,water spray,white wine,whole weat flour,whole wheat pasta,whole wheat rice,yams,yogurt cake,zucchini
0,0,1,1,0,1,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7496,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7497,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7498,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7499,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
from mlxtend.frequent_patterns import apriori

frequent_itemsets = apriori(data, min_support=0.003, use_colnames=True)
frequent_itemsets = frequent_itemsets.sort_values(by="support",ascending=False)
frequent_itemsets 

,support,itemsets
69,0.238368,(mineral water)
34,0.179709,(eggs)
96,0.174110,(spaghetti)
40,0.170911,(french fries)
23,0.163845,(chocolate)
...,...,...
992,0.003066,"(ground beef, spaghetti, cereals)"
995,0.003066,"(chicken, chocolate, eggs)"
1009,0.003066,"(chicken, spaghetti, frozen vegetables)"
1017,0.003066,"(chicken, pancakes, milk)"


In [7]:
from mlxtend.frequent_patterns import association_rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.5)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(spaghetti),(mineral water),0.174110,0.238368,0.059725,0.343032,1.439085,0.018223,1.159314
1,(mineral water),(spaghetti),0.238368,0.174110,0.059725,0.250559,1.439085,0.018223,1.102008
2,(chocolate),(mineral water),0.163845,0.238368,0.052660,0.321400,1.348332,0.013604,1.122357
3,(mineral water),(chocolate),0.238368,0.163845,0.052660,0.220917,1.348332,0.013604,1.073256
4,(eggs),(mineral water),0.179709,0.238368,0.050927,0.283383,1.188845,0.008090,1.062815
...,...,...,...,...,...,...,...,...,...
5117,(chicken),"(pancakes, milk)",0.059992,0.016531,0.003066,0.051111,3.091810,0.002075,1.036443
5118,(pancakes),"(chicken, milk)",0.095054,0.014798,0.003066,0.032258,2.179890,0.001660,1.018042
5119,(milk),"(chicken, pancakes)",0.129583,0.009065,0.003066,0.023663,2.610188,0.001892,1.014951
5120,(fresh bread),(grated cheese),0.043061,0.052393,0.003066,0.071207,1.359102,0.000810,1.020257


In [8]:
association_rules(frequent_itemsets, metric="confidence", min_threshold=0.2)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(spaghetti),(mineral water),0.174110,0.238368,0.059725,0.343032,1.439085,0.018223,1.159314
1,(mineral water),(spaghetti),0.238368,0.174110,0.059725,0.250559,1.439085,0.018223,1.102008
2,(chocolate),(mineral water),0.163845,0.238368,0.052660,0.321400,1.348332,0.013604,1.122357
3,(mineral water),(chocolate),0.238368,0.163845,0.052660,0.220917,1.348332,0.013604,1.073256
4,(eggs),(mineral water),0.179709,0.238368,0.050927,0.283383,1.188845,0.008090,1.062815
...,...,...,...,...,...,...,...,...,...
1357,"(chicken, chocolate)",(eggs),0.014665,0.179709,0.003066,0.209091,1.163495,0.000431,1.037149
1358,"(chicken, eggs)",(chocolate),0.014398,0.163845,0.003066,0.212963,1.299785,0.000707,1.062409
1359,"(chicken, frozen vegetables)",(spaghetti),0.008932,0.174110,0.003066,0.343284,1.971646,0.001511,1.257605
1360,"(chicken, pancakes)",(milk),0.009065,0.129583,0.003066,0.338235,2.610188,0.001892,1.315297


In [15]:
# tune lift to top 30
association_rules(frequent_itemsets, metric="lift", min_threshold=5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(whole wheat pasta, mineral water)",(olive oil),0.009599,0.065858,0.003866,0.402778,6.115863,0.003234,1.564145
1,(olive oil),"(whole wheat pasta, mineral water)",0.065858,0.009599,0.003866,0.058704,6.115863,0.003234,1.052168
2,"(spaghetti, frozen vegetables)","(shrimp, mineral water)",0.027863,0.023597,0.003333,0.119617,5.069202,0.002675,1.109067
3,"(shrimp, mineral water)","(spaghetti, frozen vegetables)",0.023597,0.027863,0.003333,0.141243,5.069202,0.002675,1.132028
4,"(olive oil, frozen vegetables)","(milk, mineral water)",0.011332,0.047994,0.003333,0.294118,6.128268,0.002789,1.348676
5,"(olive oil, milk)","(mineral water, frozen vegetables)",0.017064,0.035729,0.003333,0.195313,5.466564,0.002723,1.198318
6,"(olive oil, mineral water)","(milk, frozen vegetables)",0.027596,0.023597,0.003333,0.120773,5.118180,0.002682,1.110524
7,"(milk, frozen vegetables)","(olive oil, mineral water)",0.023597,0.027596,0.003333,0.141243,5.118180,0.002682,1.132338
8,"(mineral water, frozen vegetables)","(olive oil, milk)",0.035729,0.017064,0.003333,0.093284,5.466564,0.002723,1.084061
9,"(milk, mineral water)","(olive oil, frozen vegetables)",0.047994,0.011332,0.003333,0.069444,6.128268,0.002789,1.062449
